In [1]:
#Simple feature map to feed arrays into the classifier. 
import numpy as np
import json
import pickle

In [3]:
def word_shape_features(word):
    return np.array([word.istitle(), word.islower(), word.isupper(), len(word),
                     word.isdigit(),  word.isalpha(),word.isalnum(), word.isnumeric()])

def get_word_features(word):
    return word_shape_features(word)

def get_sent_features(sent):
    ret = []
    for word in sent:
        ret.append(get_word_features(word))
    return ret

In [50]:
TAGS =  pickle.load(open( "tags.pickle", "rb" ))
TAGS.remove("O") 
NUM_TAGS = len(TAGS)


In [54]:
# TAGS =  ["abc", "def"] 
# NUM_TAGS = len(TAGS)

In [55]:
tag2id = {}
for id,label in enumerate(TAGS):
    tag2id[label] = id 

def label2id(labels):
    ret = []
    prev_label = ""
    for label in labels:
        if label == "O":
            ret.append([2*NUM_TAGS])
        elif label == prev_label:
            l =[tag2id[t]+ NUM_TAGS for t in label]
            ret.append(l)
        else:
            l =[tag2id[t] for t in label]
            ret.append(l)
        prev_label = label
    return ret 


In [56]:
label2id(["O",["abc", "abc"], ["def", "abc"]])

[[4], [0, 0], [1, 0]]

In [57]:
def get_label(label_id):
    if label_id == (2*NUM_TAGS):
        return "O"
    elif label_id >= NUM_TAGS:
        return [TAGS[label_id-NUM_TAGS]]
    else:
        return TAGS[label_id]


In [58]:
def id2label(labels):
    ret = []
    for label in labels:
        l = [get_label(x) for x in label]
        if len(l) == 1 and l[0] == "O":
            l = "O"
        ret.append(l)
    return ret 


In [59]:
id2label([[4],[2,3]])

['O', [['abc'], ['def']]]

In [9]:
f = open('data/test.json')
data = json.load(f)
f.close()
for id, d in enumerate(data):
    data[id]["features"] = get_sent_features(d["sent"])
    data[id]["labels"] = get_labels(d["tags"])
    # break
    # print(d["sent"])
    # print(d["tags"])

In [ ]:
# Word identity
# word embeddings
# Word suffix (last 2 and 3 characters)
# Word prefix (first 2 and 3 characters)
# Whether the word contains a hyphen
# Whether the word contains a dot
# Whether the word contains a slash